In [371]:
import pandas as pd
import numpy as np
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
import urllib.request
from soyspacing.countbase import RuleDict, CountSpace
import torch
import math
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from kobert_transformers import get_tokenizer
import re
from collections import Counter
from ITglossary import ITGlossary,ITGlossaryUpdater
import pymysql
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm

In [416]:
# db연결
host = "database-1.cb6dvhektjhd.ap-northeast-2.rds.amazonaws.com"
conn = pymysql.connect(host=host, user='gihun', password='EM7E7e', db='production')

# 데이터 전체 조회
try:
    with conn.cursor() as cursor:
        query = "SELECT * FROM production.cosine"; # 채용공고 가져오기
        query2 = "SELECT * FROM analytics.total_stack";  # 스택 가져오기
        df = pd.read_sql_query(query, conn)
        stack_df = pd.read_sql_query(query2, conn)
finally:
    conn.close()

In [458]:
def get_words(input_text, number):
    '''
    영어 찾아주는 함수입니다.
    '''
    input_text = input_text.lower()
    words = re.findall(r'\b[a-zA-Z]+\b', input_text)
    word_counts = Counter(words)
    it_words = [word for word, count in word_counts.items() if count > number]
    return it_words
def translate_IT(input_text, it_glossary):
    '''
    it_glossary: IT 용어들 사전 딕셔너리입니다.
    text를 입력 받아서 한글로 번역해주는 함수입니다.
    '''
    input_text = str(input_text)
    words = re.findall(r'\b[a-zA-Z]+\b', input_text)
    for word in words:
        try:
            input_text = input_text.replace(word, it_glossary[word])
        except:
            pass
        input_text = input_text.replace('[','')
        input_text = input_text.replace(']','')

    return input_text
def clean_text(input_text):
    '''
    특수문자, 개행문자, HTML, www, www~, http~ 등 제거해주는 전처리 함수입니다.
    '''
    if not isinstance(input_text, str):
        return input_text
    input_text = input_text.lower()
    # 특수문자 제거
    cleaned_text = re.sub(r'[^\w\s]', '', input_text)
    cleaned_text = re.sub(r'\([^)]*\)', '', cleaned_text)
    cleaned_text = re.sub(r'\r', '', cleaned_text)
     # 개행문자 제거
    cleaned_text = cleaned_text.replace('\n', '')
    # HTML 태그 제거
    cleaned_text = re.sub(r'<[^>]+>', '', cleaned_text)
    # 텍스트에서 "www" 또는 "www~" 제거
    cleaned_text = re.sub(r'www~?.(\w+.)+\w+', '', cleaned_text)
    # 텍스트에서 "http~"제거
    cleaned_text = re.sub(r'http~?','', cleaned_text)
    return cleaned_text
def word_score(score):
    '''
    WordExtractor가 계산하는 것은 다양한 종류의 단어 가능 점수들입니다. 이를 잘 조합하여 원하는 점수를 만들 수도 있습니다. 즐겨쓰는 방법 중 하나는 cohesion_forward에 right_branching_entropy를 곱하는 것으로, 
    (1) 주어진 글자가 유기적으로 연결되어 함께 자주 나타나고, (2) 그 단어의 우측에 다양한 조사, 어미, 혹은 다른 단어가 등장하여 단어의 우측의 branching entropy가 높다는 의미입니다.
    '''
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))
def remove_english(text):
    pattern = re.compile(r'[a-zA-Z]+')
    
    text_without_english = re.sub(pattern, '', text)
    
    return text_without_english
def get_space(text):
    from pykospacing import Spacing
    spacing = Spacing()
    text = spacing(text) 
    return text
# 불용어 처리(리스트 받아서, 불용어 제거된 리스트 반환)
def stop_words_remover(list):
    return [word for word in list if not word in stopwords]
# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results
def get_embedding(keywords):
    #if not isinstance(keywords, (list, tuple)):
    #    keywords = [(keywords, 1.0)]
    embeddings = [] # 임베딩 리스트
    weights = [] # 가중치 리스트
    for keyword, weight in keywords: # 키워드와 가중치 리스트 반복
        # tensor함수로 키워드 인코딩
        input_ids = torch.tensor([tokenizer.encode(keyword)])
        with torch.no_grad():
            last_hidden_states = model(input_ids)[0]
        # 키워드 임베딩
        embedding = last_hidden_states[0].mean(dim=0).numpy()
        # 임베딩 추가
        embeddings.append(embedding)
        # 가중치 추가
        weights.append(weight)
    # 가중치가 0인 경우 Null값 추가
    if not weights or all(weight == 0 for weight in weights):
        print("가중치 Null값이 발생하였습니다.")
        return None
    return np.average(embeddings, axis=0, weights=weights)

In [374]:
glossary = ITGlossary()
updater = ITGlossaryUpdater(glossary)
updater.update_glossary()
glossary.print_glossary()

atlassian: 아틀라시안
aws: 아마존웹서비스
gitlab: 깃랩
cicd: 씨아이씨디
ai: 인공지능
api: 에이피아이
sw: 소프트웨어
nosql: 노에스큐엘
django: 장고
spring: 스프링
boot: 부트
engineer: 엔지니어
product: 제품
typescript: 타입스크립트
nestjs: 네스트제이에스
nest.js: 넥스트제이에스
python: 파이썬
git: 깃
kotlin: 코틀린
mariadb: 마리아데이터베이스
redis: 리디스
frontend: 프론트엔드
react: 리엑트
openapi: 오픈에이피아이
selenium: 셀레니움
tool: 툴
oracle: 오라클
server: 서버
kafka: 카프카
github: 깃허브
DBA: 데이터베이스엔지니어
pm: 프로젝트매니저
fullstack: 풀스택
web: 웹
mobile: 모바일
devops: 시스템엔지니어
java: 자바
javascript: 자바스크립트
jquery: 제이쿼리
postgresql: 포스트그레에스큐엘
mysql: 마이에스큐엘
restapi: 레스트에이피아이
linux: 리눅스
node.js: 노드제이에스
c++: 씨플플
reactnative: 리액트네이티브
firebase: 파이어베이스
springdatajpa: 스프링데이터제이피에이
springboot: 스프링부트
awsec2: 아마존일래스틱컴퓨트
jenkins: 젠킨스
swift: 스위프트
rubyonrails: 루비온레일즈
android: 안드로이드
php: 피에이치피
vue.js: 뷰제이에스
vuejs: 뷰제이에스
html5: 에이치티엠엘오
css3: 씨에스에스3
reactjs: 리엑트제이에스
apolloclient: 아폴로클라이언트
ubuntu: 우분투
babel: 바벨
webpack: 웹펙
sass: 사스
graphql: 그래프큐엘
nextjs: 넥스트제이에스
next.js: 넥스트제이에스
typeorm: 타이프오알엠
expressjs: 익스프레스제이에스
apolloserver: 

### User 딕셔너리

In [419]:
self_intr = users["Content"][0] # 자기소개서
self_intr = translate_IT(clean_text(self_intr), glossary)
self_intr = remove_english(self_intr)
self_intr = clean_text(self_intr)

### 띄어쓰기

In [421]:
# CountSpace 객체를 생성하고 띄어쓰기 보정을 수행합니다.
count_space = CountSpace()
count_space.load_model('./soyspacing/test.model', json_format=False)
verbose=False
mc = 10  # 보정이 적용되는 데 필요한 최소 문자 수를 나타내는 정수 값입니다.
ft = 0.3 # 강제 수정에 대한 절대 임계값을 나타내는 부동 값입니다.
nt =-0.3 # 비공간 보정의 임계값을 나타내는 부동값.
st = 0.3 # 공간 보정을 위한 임계값을 나타내는 플로트 값입니다.
# 유저 자기소개서 띄어쓰기
sent_corrected, tags = count_space.correct(
doc=self_intr,
verbose=verbose,
force_abs_threshold=ft,
nonspace_threshold=nt,
space_threshold=st,
min_count=mc)
self_intr = sent_corrected

### Kiwi 

In [422]:
# 사용자 정의 단어 추가하기
dict = {"서버":0.5,"백엔드": 0.5, "개발자": 0.5, "프론트엔드":0.5, "솔루션":0.5,"풀스택":0.5, "시스템엔지니어":0.5, "모바일":0.5, "인공지능":0.5,"게임":0.5, "임베디드":0.5, "데이터엔지니어":0.5,"클라우드엔지니어":0.5,"데이터사이언티스트":0.5,"데이터분석가":0.5, "블록체인":0.5, "기술지원":0.5,"인터넷":0.5,"보안":0.5,"머신러닝":0.3,"딥러닝":0.3,"헤이홈스퀘어": 0, "개발":0, "고퀄":0, "데이터베이스":0.3, "데이터":0.3,"소프트웨어":0.5, "알고리즘":0.3, "블록체인":0.3} 
# 키위 객체 생성
kiwi = Kiwi()
for text, score in dict.items():
    kiwi.add_user_word(text, "NNP",score)
kiwi.prepare()
# 명사 추출 빈 리스트
result4 = []
# 명사 추출해서 추가하기
# 유저 자기소개서 추출
nouns = noun_extractor(self_intr)
self_intr = " ".join(nouns)

자격 사항 3년 이상의 어플리케이션 개발 경력이 있으신 분 사용이 능숙하고 에 기반한 개발 경험이 많은 분  개발 경험이 있으신 분 논리적이고 체계적인 문제해결 능력이 있으신 분 효과적인 의사전달 및 협업 능력이 있으신 분 이공학 석사 이상 학위 소지 또는 그에 준하는 연구개발 경험이 있으신 분


### Keybert 모델

In [423]:
# 키워드 추출 모델
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

In [424]:
result6 = []
# 상위 10개 키워드와 가중치 추출하기 sky/kobert모델
keywords = kw_model.extract_keywords(self_intr, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
self_intr = keywords

In [425]:
tokenizer = get_tokenizer()
mnl_model = BertModel.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


### 임베딩하기

In [426]:
embedding = get_embedding(self_intr)

In [427]:
embedding = get_embedding(self_intr)
embedding2 = []

for s in df["embedding"]:
    if s is None:
        embedding2.append(np.zeros((768,)))
        continue
    
    s = s.replace('\n', '').replace('[', '').replace(']', '')
    s = re.split(r'\s+', s.strip())
    
    if len(s) == 0:
        embedding2.append(np.zeros((768,)))
        continue
    
    arr = np.array(s, dtype=float)
    arr = arr.reshape((768,))
    embedding2.append(arr)

In [507]:
# 유저 데이터와 post데이터의 embedding들 사이의 코사인 유사도 데이터
cos_sim = [dot(embedding, emb)/(norm(embedding)*norm(emb)) for emb in embedding2]
# 배열로 바꾸기
cos_sim = np.array(cos_sim)
# None값 0으로 바꾸기
cos_sim[np.isnan(cos_sim)] = 0
# 상위 10개 인덱스
top_indices = np.argpartition(cos_sim, -10)[-10:]
# 채용공고에서 출력
df.iloc[top_indices]

C:\Users\gihun\AppData\Local\Temp\ipykernel_7468\804626041.py:4: RuntimeWarning: invalid value encountered in scalar divide
  cos_sim = [dot(embedding, emb)/(norm(embedding)*norm(emb)) for emb in embedding2]


,id,company,position,title,stack,main_business,qualification,preferences,career_id,site,date,latitude,longitude,inner_company,address,keywords,weights,embedding
2105,2106,AB180,DevOps/시스템엔지니어,Junior DevOps Engineer(주니어 데브옵스 엔지니어)(병역특례 가능),"AWS EC2,AWS DynamoDB,Apache Druid,Python,Go",1억 대의 디바이스. 100만 RPM.\n하루 10억 건 이상의 이벤트 데이터.\n...,최소 2년 이상의 DevOps 업무 경험\n하나 이상의 개발 언어로 개발 혹은 자동...,AdTech 관련 도메인 경험\n더 적은 비용으로 시스템을 운영하기 위한 비용 관리...,3,https://career.programmers.co.kr/job_positions...,2023-06-26,37.498679,127.025042,AB180,서울 서초구 서초동 1316-31,"['전략', '비즈니스', '사용자', '회사', '효율', '업계', '기업', ...","[0.4986, 0.4978, 0.4973, 0.4923, 0.4793, 0.477...",[ 3.29797016e-01 -4.35865475e-02 2.66250595e-...
2106,2107,AB180,DevOps/시스템엔지니어,Senior DevOps Engineer(시니어 데브옵스 엔지니어)(병역특례 가능),"Go,Python,Apache Druid,AWS DynamoDB,AWS EC2",1억 대의 디바이스. 100만 RPM.\n하루 10억 건 이상의 이벤트 데이터.\n...,최소 5년 이상의 DevOps 업무 경험\n하나 이상의 개발 언어로 개발 혹은 자동...,AdTech 관련 도메인 경험\n더 적은 비용으로 시스템을 운영하기 위한 비용 관리...,6,https://career.programmers.co.kr/job_positions...,2023-06-26,37.498679,127.025042,AB180,서울 서초구 서초동 1316-31,"['전략', '비즈니스', '사용자', '회사', '효율', '업계', '기업', ...","[0.4986, 0.4978, 0.4973, 0.4923, 0.4793, 0.477...",[ 3.29797016e-01 -4.35865475e-02 2.66250595e-...
1582,1583,아드리엘(Adriel),데이터엔지니어,백엔드 엔지니어,"Node.js,ExpressJS,PostgreSQL,Redis,AWS EC2,AWS...","㈜아드리엘에서 백엔드 엔지니어는 총 4인으로 구성된 백엔드 팀에서 근무하시게 되며,...",Javascript (Node.js) 사용에 능숙하신 분\nNode.js 기반 ex...,Vue.js 와 같은 프론트엔드 프레임웍에 친숙하신 분\nAWS를 이용한 서버구성 ...,4,https://career.programmers.co.kr/job_positions...,2023-06-26,37.571838,126.982234,아드리엘(Adriel),서울 종로구 공평동 144,"['사용자', '개발자', '서비스', '디지털', '경영진', '업계', '기업'...","[0.5629, 0.553, 0.5283, 0.5258, 0.5204, 0.5187...",[ 2.98151146e-01 -4.52707147e-02 8.75177008e-...
1581,1582,아드리엘(Adriel),서버/백엔드,백엔드 엔지니어,"Node.js,ExpressJS,PostgreSQL,Redis,AWS EC2,AWS...","㈜아드리엘에서 백엔드 엔지니어는 총 4인으로 구성된 백엔드 팀에서 근무하시게 되며,...",Javascript (Node.js) 사용에 능숙하신 분\nNode.js 기반 ex...,Vue.js 와 같은 프론트엔드 프레임웍에 친숙하신 분\nAWS를 이용한 서버구성 ...,4,https://career.programmers.co.kr/job_positions...,2023-06-26,37.571838,126.982234,아드리엘(Adriel),서울 종로구 공평동 144,"['사용자', '개발자', '서비스', '디지털', '경영진', '업계', '기업'...","[0.5629, 0.553, 0.5283, 0.5258, 0.5204, 0.5187...",[ 2.98151146e-01 -4.52707147e-02 8.75177008e-...
3650,3651,오케스트로,QA,품질관리/PMO,"SW,QA,Microsoft Office 365,Confluence,Jira",• 프로젝트 관리활동(범위/일정/품질/위험/투입인력/의사소통/산출물 관리)을 기반한...,"• IT 회사 및 프로젝트의 PMO 또는 사업관리, QA 업무에 경험이 있거나 관심...",• PMP 등 프로젝트 관리 자격을 보유하신 분\n• 기능점수(FP) 산정 유경험 ...,1,https://www.jumpit.co.kr/position/12817,2023-06-26,37.525191,126.929113,오케스트로 영등포,서울 영등포구 여의도동 22,"['회사', '관리', '프로젝트', '책임자', '협업', '조직', '모니터링'...","[0.5484, 0.5374, 0.521, 0.5165, 0.5025, 0.4941...",[ 3.04660813e-01 -7.45223951e-02 7.91941106e-...
3740,3741,오케스트로,개발PM,품질관리/PMO,"SW,QA,Microsoft Office 365,Confluence,Jira",• 프로젝트 관리활동(범위/일정/품질/위험/투입인력/의사소통/산출물 관리)을 기반한...,"• IT 회사 및 프로젝트의 PMO 또는 사업관리, QA 업무에 경험이 있거나 관심...",• PMP 등 프로젝트 관리 자격을 보유하신 분\n• 기능점수(FP) 산정 유경험 ...,1,https://www.jumpit.co.kr/position/12817,2023-06-26,37.525191,126.929113,오케스트로 영등포,서울 영등포구 여의도동 22,"['회사', '관리', '프로젝트', '책임자', '협업', '조직', '모니터링'...","[0.5484, 0.5374, 0.521, 0.5165, 0.5025, 0.4941...",[ 3.04660813e-01 -7.45223951e-02 7.91941106e-...
2091,2092,한국클라우드,인공지능/머신러닝,공공정책사업실 R&D 개발자,"Python,Java",1. (Product Management) 상품/서비스 기획 :\n- 신규 서비스...,1. 최신 IT 기술 및 상품/서비스 기획에 대한 이해가 높으신 분\n2. 문제 정...,"1. 시장조사 및 사업검토, 사업 추진계획서 등 기획 자료 작성 경험이 있으신 분\...",1,https://career.programmers.co.kr/job_positions...,2023-06-26,37.479953,126.877008,한국클라우드,서울 금천구 가산동 543-1,"['캠페인', '정부', '조직', '서비스', '전략', '업계', '제안서', ...","[0.4698, 0.4472, 0.439, 0.4359, 0.4354, 0.4305...",[ 3.09467506e-01 -9.71278559e-02 3.44715281e-...
1583,1584,아드리엘(Adriel),DevOps/시스템엔지니어,백엔드 엔지니어,"Node.js,ExpressJS,PostgreSQL,Redis,AWS EC2,AWS...","㈜아드리엘에서 백엔드 엔지니어는 총 4인으로 구성된 백엔드 팀에서 근무하시게 되며,...",Javascript (Node.js) 사용에 능숙하신 분\nNode.js 기반 ex...,Vue.js 와 같은 프론트엔드 프레임웍에 친숙하신 분\nAWS를 이용한 서버구성 ...,4,https://career.programmers.co.kr/job_positions...,2023-06-26,37.571838

In [535]:
# 키워드 바꾸기
keywords_str = []
for k in df["keywords"]:
    print(k)
    keywords_str.append(eval(k))

# 가중치 바꾸기
weights_str = []
for w in df["weights"]:
    weights_list = [float(x) for x in eval(w)]
    weights_str.append(weights_list)
keywords = [keyword for keyword, _ in keywords_str]
weights = [weight for _, weight in weights_str]

# Convert weights to float values
weights_float = [float(weight) for weight in weights]

# Create a dictionary from keywords and weights
word_freq = {k: w for k, w in zip(keywords, weights_float)}

# Generate the word cloud
wordcloud = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(word_freq)

# Plot the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Topic Keywords Word Cloud')
plt.show()

['개발', '경험', '소프트웨어', '교육', '기술', '연구', '경력', '구현', '이공학', '설계']
['경험', '알고리즘', '개발자', '데이터', '프로세싱', '개발', '기술', '연구', '이공학', '구현']
['알고리즘', '머신러닝', '프로세싱', '기술', '이공학', '데이터', '개발', '모델', '딥러닝', '분석']
['알고리즘', '머신러닝', '프로세싱', '기술', '이공학', '데이터', '개발', '모델', '딥러닝', '분석']
['알고리즘', '머신러닝', '프로세싱', '기술', '이공학', '데이터', '개발', '모델', '딥러닝', '분석']
['경험', '모바일', '개발', '카드', '서비스', '운영', '활용', '모듈', '배포', '경력']
['경험', '모바일', '개발', '카드', '서비스', '운영', '활용', '모듈', '배포', '경력']
['개발', '경험', '프로그래밍', '서비스', '디자인', '인터페이스', '경력', '운영', '언어', '배포']
['개발', '프로그래밍', '경험', '인터페이스', '디자인', '서비스', '운영', '언어', '배포', '적용']
['개발', '소프트웨어', '프로토콜', '지원자', '기술', '서비스', '공학', '자격증', '지식', '인프라']
['서버', '데이터베이스', '시스템', '체제', '리눅스', '운영', '보안', '지식', '프로젝트', '기반']
['경력', '서버', '경력자', '공학', '플랫폼', '설계', '관리', '개발', '운영', '커뮤니케이션']
['경력', '서버', '경력자', '공학', '플랫폼', '설계', '관리', '개발', '운영', '커뮤니케이션']
['경력', '서버', '경력자', '공학', '플랫폼', '설계', '관리', '개발', '운영', '커뮤니케이션']
['프로젝트', '서비스', '기술', '개발', '협업', '운영', '디자인', '기반', 

TypeError: eval() arg 1 must be a string, bytes or code object